# TP 3 - NLP - Embeddings


<img src="https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/logoFIUBA.jpg" width="500" align="center">


### Alumno: Domenje Carlos R.

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.


### Librerias a utilizar

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os

import multiprocessing
from gensim.models import Word2Vec

# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np        

In [2]:
# Lectura del dataset
path_base = "./data/"
file_name = os.listdir(path_base)
file_name

['blink-182.txt']

In [3]:
file = open(path_base+file_name[0], mode="rt", encoding='utf-8')
text=file.read()
file.close()

text

'Can we forget about the things I said when I was drunk...\nI didn\'t mean to call you that\nI can\'t remember what was said\nOr what you threw at me Please tell me\nPlease tell me why\nMy car is in the front yard\nAnd I am sleeping with my cloths on\nI came in throught the window... Last night\nAnd your... Gone\nGone It\'s no suprise to me I am my own worst enemy\nCuz every now and then I kick the living shit out of me\nThe smoke alarm is going offf and there a cigarette\nStill buring Please tell me why\nMy car is in the front yard\nAnd I\'m sleeping with my clothes on\nI came in throught the windo last night\nAnd your gone\nGone Please tell me why\nMy car is in the front yard\nAnd I am sleeping with my clothes on\nI came inthrough the windo last night It\'s no suprise to me\nI am my own worst enemy\nCuz every now and then I kick the livin\' shit out of me Can we forget about the things I said when I was drunk\nI didn\'t mean to call you that I, I can\'t get you more pregnant\nbut if 

#### Separación de texto por párrafos - Eliminación de vacios.

In [4]:
txt_list = text.split("\n")
txt_list = list(filter(None, txt_list))
txt_list

['Can we forget about the things I said when I was drunk...',
 "I didn't mean to call you that",
 "I can't remember what was said",
 'Or what you threw at me Please tell me',
 'Please tell me why',
 'My car is in the front yard',
 'And I am sleeping with my cloths on',
 'I came in throught the window... Last night',
 'And your... Gone',
 "Gone It's no suprise to me I am my own worst enemy",
 'Cuz every now and then I kick the living shit out of me',
 'The smoke alarm is going offf and there a cigarette',
 'Still buring Please tell me why',
 'My car is in the front yard',
 "And I'm sleeping with my clothes on",
 'I came in throught the windo last night',
 'And your gone',
 'Gone Please tell me why',
 'My car is in the front yard',
 'And I am sleeping with my clothes on',
 "I came inthrough the windo last night It's no suprise to me",
 'I am my own worst enemy',
 "Cuz every now and then I kick the livin' shit out of me Can we forget about the things I said when I was drunk",
 "I didn't m

In [5]:
# Armar el dataset
df  = pd.DataFrame(txt_list)
df.head()

,0
0,Can we forget about the things I said when I w...
1,I didn't mean to call you that
2,I can't remember what was said
3,Or what you threw at me Please tell me
4,Please tell me why


In [6]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 2332


### Preprocesamiento

In [7]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [8]:
# Vemos los tokens de algunas oraciones
sentence_tokens[:2]

[['can',
  'we',
  'forget',
  'about',
  'the',
  'things',
  'i',
  'said',
  'when',
  'i',
  'was',
  'drunk'],
 ['i', "didn't", 'mean', 'to', 'call', 'you', 'that']]

#### Word2Vec

In [9]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobracargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [10]:
# Crearmos el modelo generador de vectores con skigram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [11]:
w2v_model_cbow = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=0)           # modelo 0:CBOW  1:skipgram

In [12]:
# Buildeamos el vocabulario con los tokens para los 2 modelos
w2v_model.build_vocab(sentence_tokens)
w2v_model_cbow.build_vocab(sentence_tokens)

In [13]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 2332


In [14]:
# Cantidad de words encontradas en el corpus words
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 567


#### Entrenamiento

In [15]:
# Entrenamos el modelo generador de vectores con skigram
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=400,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 170263.125
Loss after epoch 1: 100812.25
Loss after epoch 2: 99248.28125
Loss after epoch 3: 97995.375
Loss after epoch 4: 97418.53125
Loss after epoch 5: 98082.8125
Loss after epoch 6: 95027.5625
Loss after epoch 7: 90131.25
Loss after epoch 8: 87438.5625
Loss after epoch 9: 82128.375
Loss after epoch 10: 74934.5
Loss after epoch 11: 70057.5
Loss after epoch 12: 67097.25
Loss after epoch 13: 64852.625
Loss after epoch 14: 63975.625
Loss after epoch 15: 61814.0
Loss after epoch 16: 61287.25
Loss after epoch 17: 59780.5
Loss after epoch 18: 59314.75
Loss after epoch 19: 58607.375
Loss after epoch 20: 57476.0
Loss after epoch 21: 56536.875
Loss after epoch 22: 56296.375
Loss after epoch 23: 56174.625
Loss after epoch 24: 55740.25
Loss after epoch 25: 55347.5
Loss after epoch 26: 54416.125
Loss after epoch 27: 53708.0
Loss after epoch 28: 52273.5
Loss after epoch 29: 52202.25
Loss after epoch 30: 51015.25
Loss after epoch 31: 50849.0
Loss after epoch 32: 50979.5
Loss a

(4577819, 7900400)

In [16]:
# Entrenamos el modelo generador de vectores con cbow
w2v_model_cbow.train(sentence_tokens,
                 total_examples=w2v_model_cbow.corpus_count,
                 epochs=400,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 97680.3515625
Loss after epoch 1: 50490.1328125
Loss after epoch 2: 48166.734375
Loss after epoch 3: 47505.578125
Loss after epoch 4: 47308.390625
Loss after epoch 5: 47049.78125
Loss after epoch 6: 46985.25
Loss after epoch 7: 45682.21875
Loss after epoch 8: 45328.53125
Loss after epoch 9: 43912.0625
Loss after epoch 10: 42901.21875
Loss after epoch 11: 41559.3125
Loss after epoch 12: 39741.625
Loss after epoch 13: 38043.625
Loss after epoch 14: 36902.9375
Loss after epoch 15: 35452.75
Loss after epoch 16: 34037.375
Loss after epoch 17: 32845.5625
Loss after epoch 18: 31745.1875
Loss after epoch 19: 30751.375
Loss after epoch 20: 29799.75
Loss after epoch 21: 28889.375
Loss after epoch 22: 28041.1875
Loss after epoch 23: 27264.9375
Loss after epoch 24: 26712.75
Loss after epoch 25: 26200.125
Loss after epoch 26: 24225.375
Loss after epoch 27: 23878.5
Loss after epoch 28: 23500.25
Loss after epoch 29: 23292.25
Loss after epoch 30: 22383.125
Loss after epoch 31: 2229

(4577819, 7900400)

#### Ensayos

In [17]:
wordToFind = "home"
print("Palabras que mas se relacionan con "+wordToFind+ " en skip-gram")
w2v_model.wv.most_similar(positive=[wordToFind], topn=10)

Palabras que mas se relacionan con home en skip-gram


[('matt', 0.5106375217437744),
 ('skiba', 0.4911095201969147),
 ('midnight', 0.4731724262237549),
 ('hoppus', 0.44728803634643555),
 ('carry', 0.41197463870048523),
 ('slowly', 0.4018676280975342),
 ('los', 0.3786796033382416),
 ('under', 0.3676787316799164),
 ('today', 0.3660873770713806),
 ('breath', 0.36201316118240356)]

In [18]:
print("Palabras que mas se relacionan con "+wordToFind+ " en CBOW")
w2v_model_cbow.wv.most_similar(positive=[wordToFind], topn=10)

Palabras que mas se relacionan con home en CBOW


[('slowly', 0.26489725708961487),
 ('while', 0.24244476854801178),
 ('under', 0.2393859624862671),
 ('long', 0.23736417293548584),
 ("you'll", 0.22665250301361084),
 ('dying', 0.22577504813671112),
 ('today', 0.22431887686252594),
 ('house', 0.223989799618721),
 ('holly', 0.22160682082176208),
 ('skiba', 0.21653227508068085)]

In [19]:
wordToFind = "night"
print("Palabras que menos se relacionan con "+wordToFind+ " en skip-gram")
w2v_model.wv.most_similar(negative=[wordToFind], topn=10)

Palabras que menos se relacionan con night en skip-gram


[('two', 0.03217720985412598),
 ('la', 0.016151079908013344),
 ('forget', 0.007451909594237804),
 ('sure', 0.005949738435447216),
 ('miss', -0.007472590077668428),
 ('want', -0.010392986238002777),
 ('loud', -0.012289684265851974),
 ('im', -0.012353014200925827),
 ('beautiful', -0.015023616142570972),
 ('falling', -0.020136741921305656)]

In [20]:
print("Palabras que menos se relacionan con "+wordToFind+ " en CBOW")
w2v_model_cbow.wv.most_similar(negative=[wordToFind], topn=10)

Palabras que menos se relacionan con night en CBOW


[('sure', 0.3101706802845001),
 ('want', 0.26608583331108093),
 ('beautiful', 0.23888510465621948),
 ('dreaming', 0.20870254933834076),
 ('everything', 0.1801607608795166),
 ('death', 0.17185436189174652),
 ('bent', 0.1718403846025467),
 ('waiting', 0.17168934643268585),
 ('find', 0.16751471161842346),
 ('falling', 0.16525506973266602)]

In [21]:
print("Palabras que mas se relacionan con "+wordToFind+ " en skip-gram")
w2v_model.wv.most_similar(positive=[wordToFind], topn=10)

Palabras que mas se relacionan con night en skip-gram


[('online', 0.3960397243499756),
 ('gave', 0.3583752512931824),
 ('control', 0.3543393909931183),
 ("we've", 0.35147252678871155),
 ('tits', 0.3471459448337555),
 ('reach', 0.33575862646102905),
 ('fire', 0.33326056599617004),
 ('christmas', 0.3250057101249695),
 ('street', 0.3178996443748474),
 ('believe', 0.31620118021965027)]

In [22]:
print("Palabras que mas se relacionan con "+wordToFind+ " en CBOW")
w2v_model_cbow.wv.most_similar(positive=[wordToFind], topn=10)

Palabras que mas se relacionan con night en CBOW


[('gave', 0.25584548711776733),
 ('make', 0.2528972625732422),
 ('smoke', 0.243977889418602),
 ("let's", 0.24035297334194183),
 ('these', 0.23667550086975098),
 ('day', 0.22536851465702057),
 ("we've", 0.21663783490657806),
 ('star', 0.21591374278068542),
 ('torn', 0.21535533666610718),
 ('standing', 0.20976196229457855)]

In [23]:

wordToFind = "lights"
print("Palabras que mas se relacionan con "+wordToFind+ " en skip-gram")
w2v_model.wv.most_similar(positive=[wordToFind], topn=10)

Palabras que mas se relacionan con lights en skip-gram


[('fluorescent', 0.6010627746582031),
 ('underneath', 0.5082883238792419),
 ('carry', 0.4613956809043884),
 ('loose', 0.4498332738876343),
 ('songs', 0.4449063837528229),
 ('radio', 0.43798497319221497),
 ('streets', 0.42232194542884827),
 ('friend', 0.4179946780204773),
 ('midnight', 0.4028770625591278),
 ('bring', 0.40246516466140747)]

In [24]:
print("Palabras que mas se relacionan con "+wordToFind+ " en CBOW")
w2v_model_cbow.wv.most_similar(positive=[wordToFind], topn=10)

Palabras que mas se relacionan con lights en CBOW


[('radio', 0.3702556788921356),
 ('loose', 0.358029305934906),
 ('ground', 0.3428971469402313),
 ('sad', 0.31895166635513306),
 ('street', 0.3060339689254761),
 ('edge', 0.29853948950767517),
 ('kick', 0.2946494519710541),
 ('truth', 0.2938710153102875),
 ('floor', 0.28894802927970886),
 ('into', 0.2749479413032532)]

In [25]:
wordToFind = "things"
print("Palabras que menos se relacionan con "+wordToFind+ " en skip-gram")
w2v_model.wv.most_similar(negative=[wordToFind], topn=10)

Palabras que menos se relacionan con things en skip-gram


[('without', 0.006927588488906622),
 ("let's", 0.0017258005682379007),
 ('hold', -0.014423091895878315),
 ('smile', -0.01886095106601715),
 ("i've", -0.030640067532658577),
 ('two', -0.031962800770998),
 ('drink', -0.03267841041088104),
 ('save', -0.03476893529295921),
 ('or', -0.036411914974451065),
 ('falling', -0.03814849257469177)]

In [26]:
print("Palabras que menos se relacionan con "+wordToFind+ " en CBOW")
w2v_model_cbow.wv.most_similar(negative=[wordToFind], topn=10)

Palabras que menos se relacionan con things en CBOW


[('two', 0.2288517951965332),
 ('whoa', 0.22285394370555878),
 ('growing', 0.21268002688884735),
 ('say', 0.2093772143125534),
 ('la', 0.20179617404937744),
 ('without', 0.20043690502643585),
 ("i've", 0.19801731407642365),
 ('falling', 0.1951753944158554),
 ('dying', 0.19316208362579346),
 ("let's", 0.18568238615989685)]

#### Agrupación de vectores

In [27]:
def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, perplexity=15,random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [30]:
x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=350
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS],
                 title="Representacion en 2 dim de T-SNE para modelo Skip-Gram")
fig.show()

/opt/miniconda3/envs/nlp-env/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:780: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/opt/miniconda3/envs/nlp-env/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:790: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



In [31]:
x_vals, y_vals, labels = reduce_dimensions(w2v_model_cbow)

MAX_WORDS=350
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS],
                 title="Representacion en 2 dim de T-SNE para modelo CBOW")
fig.show()

/opt/miniconda3/envs/nlp-env/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:780: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/opt/miniconda3/envs/nlp-env/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:790: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



### Conclusion

Se realizó la generacion de un embedding para un corpus basado en lo visto en clases y con la ayuda del colab provisto en la práctica.

Se realizaron pruebas con CBOW y con el SKIP-GRAM utilizando la libreria GENSIM.

Se utilizó un dataset similar al mostrado en la catedra cambiando la cancion a la del grupop blink-182, se pudo ver que los resultados obtenidos con skip-gram fueron los mejores, esta inferencia se basa en las mejores relaciones obtenidas en la letra de la cancion, mas alla que en ambos algoritmos se pudieron observar resultados positivos. 
